In [3]:
'''
Simple LSTM for adding 2 numbers in binary
'''

import numpy as np
import copy

n_samples = 10001
np.random.seed(0)


input_dim = 2
output_dim = 1
n_bit = 8
hidden_size = 16
learning_rate = .1

largest_input_number = pow(2, n_bit) / 2

# weights input values -> decide what are the values calculated as a function of current inputs and outputs from previous hidden layer
w_hidden_input = np.random.standard_normal(size=(input_dim, hidden_size))
w_previous_lstm_cell_output = np.random.standard_normal(size=(hidden_size, hidden_size))

# weights input gate -> decide what information is kept as a function of current inputs and outputs from previous hidden layer
w_hidden_input_gate = np.random.standard_normal(size=(input_dim, hidden_size))
w_previous_lstm_cell_input_gate = np.random.standard_normal(size=(hidden_size, hidden_size))

# weights forget gate -> decide what information is not kept as a function of current inputs and outputs from previous hidden layer
w_hidden_forget_gate = np.random.standard_normal(size=(input_dim, hidden_size))
w_previous_lstm_cell_forget_gate = np.random.standard_normal(size=(hidden_size, hidden_size))

# weights output gate -> decide what information is not kept as a function of current inputs and outputs from previous hidden layer
w_lstm_cell_output_gate = np.random.standard_normal(size=(input_dim, hidden_size))
w_previous_lstm_cell_output_gate = np.random.standard_normal(size=(hidden_size, hidden_size))

#  weights output layer
weights_output = np.random.standard_normal(size=(hidden_size, output_dim))

def sigmoid(x): return (1 / (1 + np.exp(-x)))
def sigmoid_derivative(x): return x * (1 - x)
def tanh(x): return (np.exp(x) - np.exp(-x))  / (np.exp(x) + np.exp(-x)) 
def tanh_derivative(x): return 1-tanh(x)^2


batch_error = 0

# online learning: network gets updated with each sample on the way
for i in range(n_samples):

    # generate 2 random numbers and their sum
    input_1, input_2 = np.random.randint(0, largest_input_number), np.random.randint(0, largest_input_number)
    true_output = input_1 + input_2
    
    # calculate the binaries
    input_1_binary, input_2_binary, true_output_binary = [int(x) for x in np.binary_repr(input_1, n_bit)], [int(x) for x
                                in np.binary_repr(input_2, n_bit)], [int(x) for x in np.binary_repr(true_output, n_bit)]

    # we'll append the outputs at each layer on the way..
    lstm_cell_output_seq = []
    hidden_layer_filtered_values_seq = []
    output_layer_output_seq = []
    
    lstm_cell_output_seq.append(np.zeros((1,hidden_size)))
    hidden_layer_filtered_values_seq.append(np.zeros((1,hidden_size)))

    # forward pass of the bit sequence through the network and accumulating the errors at each bit position
    for bit_idx in range(n_bit - 1, -1, -1):
        
        input_bits = np.array([[input_1_binary[bit_idx], input_2_binary[bit_idx]]])
        
        # feed forward through LSTM cell
        hidden_layer_raw_values = tanh(np.dot(input_bits, w_hidden_input) + np.dot(lstm_cell_output_seq[-1], w_previous_lstm_cell_output))
        hidden_layer_input_gate = sigmoid(np.dot(input_bits, w_hidden_input_gate) + 
                                          np.dot(lstm_cell_output_seq[-1], w_previous_lstm_cell_input_gate)) 
        hidden_layer_forget_gate = sigmoid(np.dot(input_bits, w_hidden_forget_gate) + 
                                           np.dot(lstm_cell_output_seq[-1], w_previous_lstm_cell_forget_gate))
        
        hidden_layer_filtered_values = tanh(hidden_layer_raw_values*hidden_layer_input_gate + 
                                            hidden_layer_filtered_values_seq[-1]*w_hidden_forget_gate)  
    
        lstm_cell_output_gate = sigmoid(np.dot(input_bits, w_lstm_cell_output_gate) + 
                                        np.dot(lstm_cell_output_seq[-1], w_previous_lstm_cell_output_gate))
           
        lstm_cell_output = hidden_layer_filtered_values*lstm_cell_output_gate

        hidden_layer_filtered_values_seq.append(copy.deepcopy(hidden_layer_filtered_values))
        lstm_cell_output_seq.append(copy.deepcopy(lstm_cell_output))
        
        # feed forward through output layer
        output_layer_output = sigmoid(np.dot(lstm_cell_output, weights_output))
        output_layer_output_seq.append(copy.deepcopy(output_layer_output))
        
    
    #previous_hidden_layer_error_weighted_derivative = np.zeros((1,hidden_size))
    # append one more zero array for going backwards
    
    #sum of the derivative of the outputs at the corresponding layers weighted by the errors, for each pair of input bits
    #sum_hidden_layer_updates = np.zeros_like(weights_hidden)
    #sum_previous_hidden_layer_updates = np.zeros_like(weights_previous_hidden)
    #sum_output_previous_hidden_layer_error_weighted_derivativelayer_updates = np.zeros_like(weights_output)

    # rolling back from the last bit to the first
    hidden_layer_filtered_values_seq.reverse()
    output_layer_output_seq.reverse()
    lstm_cell_output_seq.reverse()
    
    for bit_idx in range(n_bit):
               
        # take output error at this position -> size(output_dim)
        output_error = np.array([true_output_binary[bit_idx]]) - output_layer_output_seq[bit_idx]

        # calculate output derivative weighted by the output errors -> size(output_dim) or what we call delta
        output_error_weighted_derivative = sigmoid_derivative(output_layer_output_seq[bit_idx])* output_error
        
        # sum the output_error_weighted_derivative for each element in the sequence, weighted by the size of the inputs 
            # size -> (input_dim, hidden_size)
        sum_hidden_layer_updates += np.dot(np.array([[input_1_binary[bit_idx], input_2_binary[bit_idx]]]).T, hidden_error_weighted_derivative)
        
        # propagate output layer error to lstm cell error - this error was sent to (and therefore coming back from):
            # 1. output layer - or next layer
            # 2. next hidden layer raw values
            # 3. next hidden layer filters / decisions: input gate & forget gate
            # 4. next lstm output gate decision
        # no derivatites to compute for the lstm cell output as it is not parameterized by any weights
        
        lstm_cell_error = np.dot(output_error_weighted_derivative, weights_output.T) + 
                            np.dot(previous_hidden_raw_values_error_weighted_derivative, w_previous_lstm_cell_output) +
                            np.dot(previous_hidden_input_gate_error_weighted_derivative, w_hidden_input_gate) +
                            np.dot(previous_hidden_forget_gate_error_weighted_derivative, w_hidden_forget_gate) +
                            np.dot(previous_lstm_cell_output_gate_error_weighted_derivative, w_lstm_cell_output_gate)
                        

        # propagate lstm cell error to next lstm output gate decision - this error was sent to: current lstm_cell_output only
        lstm_cell_output_gate_error = lstm_cell_error * hidden_layer_filtered_values_seq[bit_idx] * sigmoid_derivative(x)
            
        
        # propagate lstm cell error to next filtered hidden layer output - this error was sent to:
        # 1. next filtered hidden layer output - when deciding what to forget out of it
        # 2. current lstm_cell_output
        
        # propagate lstm cell error to next hidden layer filter gates - these errors were sent to current filtered hidden layer output
        
        # propagate lstm cell error + 
        
        
        '''
        
        # sum the output_error_weighted_derivative for each element in the sequence weighted by the size of inputs into this layer 
        sum_output_layer_updates += np.dot(hidden_layer_output_seq[bit_idx].T, output_error_weighted_derivative)

        # calculate hidden error as coming from: 1.what was sent to the output, 2.what was sent to the next hidden layer
        hidden_error = np.dot(output_error_weighted_derivative, weights_output.T) + 
            np.dot(previous_hidden_layer_error_weighted_derivative, weights_previous_hidden)

        # calculate hidden outputs derivatives weighted by hidden errors ->(hidden_size) * (hidden_size) = (hidden_size)
        hidden_error_weighted_derivative = sigmoid_derivative(hidden_layer_output_seq[bit_idx])* hidden_error

        # sum the output_error_weighted_derivative for each element in the sequence, weighted by the size of the inputs -> (input_dim, hidden_size)
        sum_hidden_layer_updates += np.dot(np.array([[input_1_binary[bit_idx], input_2_binary[bit_idx]]]).T, hidden_error_weighted_derivative)

        # sum the hidden_error_weighted_derivative for each element in the sequence, weighted by the size of the inputs -> (hidden_size, hidden_size)
        sum_previous_hidden_layer_updates += np.dot(hidden_layer_output_seq[bit_idx - 1].T, hidden_error_weighted_derivative)
        
        # propagating the hidden layer error back to
        previous_hidden_layer_error_weighted_derivative = hidden_error_weighted_derivative
        
        # just accumulating error for printing
        batch_error += abs(output_error[0])

       
        

    # updating weights for this sample
    weights_hidden += (sum_hidden_layer_updates * learning_rate)
    weights_previous_hidden += (sum_previous_hidden_layer_updates * learning_rate)
    weights_output += (sum_output_layer_updates * learning_rate)
    
    errors = np.array(true_output_binary) - np.array([x.tolist()[0][0] for x in output_layer_output_seq])
    batch_error += sum([abs(x) for x in errors])/n_bit
    
    if (i % 1000) == 0: 
        print 100*'#' + " sample {} ".format(i)
        print " Training sample: {0} + {1} = {2}".format(input_1, input_2, true_output)
        #print " Binary version: {0} + {1} = {2}".format(input_1_binary, input_2_binary, true_output_binary)
        result = [x.tolist()[0][0] for x in output_layer_output_seq]
        print " Result is {}".format( sum([pow(2,n_bit-i-1)*round(result[i]) for i in range(n_bit)]))
        #print result
        
        print " Average binarry error for this batch is {}".format(batch_error/8000)   
        batch_error = 0
    '''


#################################################################################################### sample 0 
 Training sample: 5 + 3 = 8
 Result is 1.0
 Average binarry error for this batch is [ 0.00031903]
#################################################################################################### sample 1000 
 Training sample: 5 + 2 = 7
 Result is 3.0
 Average binarry error for this batch is [ 0.31124201]
#################################################################################################### sample 2000 
 Training sample: 5 + 5 = 10
 Result is 0.0
 Average binarry error for this batch is [ 0.3066118]
#################################################################################################### sample 3000 
 Training sample: 6 + 4 = 10
 Result is 6.0
 Average binarry error for this batch is [ 0.29437916]
#################################################################################################### sample 4000 
 Training sample: 2 + 4 = 6
 Result is 4